## Setup & Imports

In [3]:
import pandas as pd
import boto3
import json
from io import StringIO

# Configure S3
s3_client = boto3.client('s3', region_name='us-east-1')
BUCKET_NAME = 'akshays3-2026'

print("Setup complete")

Setup complete


/Users/akshay/Desktop/GitHub/rearcinfra/.venv/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## Load Data from S3

In [6]:
# Load BLS data
bls_key = 'raw/bls/pr.data.0.Current'
obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=bls_key)
bls_content = obj['Body'].read().decode('utf-8')

bls_df = pd.read_csv(StringIO(bls_content), sep='\t', dtype=str)
bls_df.columns = bls_df.columns.str.strip()
bls_df = bls_df.map(lambda x: x.strip() if isinstance(x, str) else x)
bls_df['value'] = pd.to_numeric(bls_df['value'], errors='coerce')
bls_df['year'] = pd.to_numeric(bls_df['year'], errors='coerce')

print(f"BLS data shape: {bls_df.shape}")
bls_df.head()

BLS data shape: (37239, 5)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [ ]:
# Load population data
pop_key = 'raw/datausa/population.json'
obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=pop_key)
pop_content = obj['Body'].read().decode('utf-8')
pop_json = json.loads(pop_content)

pop_rows = pop_json['data']  # DataUSA response stores records as a list under 'data'
pop_df = pd.DataFrame(pop_rows)
pop_df['Year'] = pd.to_numeric(pop_df['Year'], errors='coerce')
pop_df['Population'] = pd.to_numeric(pop_df['Population'], errors='coerce')

print(f"Population data shape: {pop_df.shape}")
pop_df.head()

TypeError: list indices must be integers or slices, not str

## Query 1: Population Statistics (2013-2018)

Calculate mean and standard deviation of annual US population for years 2013-2018 inclusive.

In [ ]:
# Filter years 2013-2018
pop_filtered = pop_df[(pop_df['Year'] >= 2013) & (pop_df['Year'] <= 2018)]

mean_pop = pop_filtered['Population'].mean()
std_pop = pop_filtered['Population'].std()

print(f"Mean Population (2013-2018): {mean_pop:,.0f}")
print(f"Standard Deviation: {std_pop:,.2f}")

pop_filtered[['Year', 'Population']]

## Query 2: Best Year per Series

For every series_id, find the year with the maximum sum of quarterly values.

In [ ]:
# Filter quarterly periods
quarterly = bls_df[bls_df['period'].str.startswith('Q', na=False)].copy()

# Group by series_id and year, sum values
grouped = quarterly.groupby(['series_id', 'year'])['value'].sum().reset_index()

# Find max year for each series
best_years = grouped.loc[grouped.groupby('series_id')['value'].idxmax()]
best_years = best_years.sort_values('series_id')

print(f"Total series analyzed: {len(best_years)}")
best_years.head(20)

## Query 3: Join Series with Population

For series_id = PRS30006032 and period = Q01, join with population data by year.

In [ ]:
# Filter BLS data
filtered_bls = bls_df[
    (bls_df['series_id'] == 'PRS30006032') & 
    (bls_df['period'] == 'Q01')
].copy()

# Join with population
merged = filtered_bls.merge(
    pop_df,
    left_on='year',
    right_on='Year',
    how='left'
)

result = merged[['series_id', 'year', 'period', 'value', 'Population']]

print(f"Total matching records: {len(result)}")
result

## Summary

All three queries executed successfully:
- Query 1: Population statistics calculated
- Query 2: Best year per series identified
- Query 3: Series joined with population data